# Market Making (RL)

*In this notebook, we will implement a simple market making strategy using reinforcement learning. The goal of the market maker is to maximize the profit by providing liquidity to the market. The market maker will place two limit orders, one buy order and one sell order, around the mid price. The market maker will adjust the price of the limit orders based on the current inventory and the recent price movements. The market maker will also adjust the size of the limit orders based on the current inventory and the recent price movements. The market maker will use reinforcement learning to learn the optimal price and size of the limit orders.*

## Setup

In [1]:
# Data loading and preprocessing
import numpy as np
import pandas as pd
import lakeapi
import datetime

# Data visualization
import plotly
import cufflinks

cufflinks.go_offline()



c:\Users\artco\Downloads\RL - Market Making\.venv\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## Data Preparation
The data used for this notebook is the [Crypto Lake website](https://crypto-lake.com/). The data contains the order book data for the "BTC/USDT" trading pair on the Binance exchange. The data contained up to 20 bid and 20 ask orders for each timestamp(~850k). The data contains the following columns:
- `timestamp`: The timestamp of the order book data.
- `bid_price_1`: The price of the highest bid order.
- `bid_size_1`: The size of the highest bid order.
- `ask_price_1`: The price of the lowest ask order.
- `ask_size_1`: The size of the lowest ask order.


### Data Loading

In [6]:
lakeapi.use_sample_data(anonymous_access = True)
books = lakeapi.load_data(
    table="book",
    start=datetime.datetime(2022, 10, 1),
    end=datetime.datetime(2022, 10, 2),
    symbols=["BTC-USDT"],
    exchanges=["BINANCE"],
)

# data cleaning
books.rename(columns = {'received_time' : 'timestamp'}, inplace = True)
books.set_index('timestamp', inplace = True)
books.drop(columns = ['exchange', 'symbol','origin_time', 'sequence_number'], inplace = True)


# check for missing values in the data
print("Missing values : ", books.isnull().sum().sum(),'\n'+ '-'*50)
# per column
# print("missing values per colum :\n", books.isnull().sum()) 


books[['bid_0_price', 'ask_0_price']][:2000].iplot(title='BTC-USDT Bid/Ask Prices', xTitle='Time', yTitle='Price', mode='lines')

Missing values :  0 
--------------------------------------------------


c:\Users\artco\Downloads\RL - Market Making\.venv\Lib\site-packages\cufflinks\plotlytools.py:117: FutureWarning:

DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.



### Data preprocessing